# Create Tables

In [3]:
--Jascharan Singh

USE QueensClassSchedule
GO
-- Create the Department table
CREATE TABLE Department (
    DepartmentID INT IDENTITY(1,1) PRIMARY KEY,
    DepartmentName NVARCHAR(100) NOT NULL,
    CreatedDate DATETIME DEFAULT (SYSDATETIME())
);

-- Create the Instructor table
CREATE TABLE Instructor (
    InstructorID INT IDENTITY(1,1) PRIMARY KEY,
    FirstName NVARCHAR(50) NOT NULL,
    LastName NVARCHAR(50) NOT NULL,
    DepartmentID INT NOT NULL,
    FOREIGN KEY (DepartmentID) REFERENCES Department(DepartmentID)
);

-- Create the Course table
CREATE TABLE Course (
    CourseID INT IDENTITY(1,1) PRIMARY KEY,
    CourseName NVARCHAR(100) NOT NULL,
    Credits INT NOT NULL
);

-- Create the Class table
CREATE TABLE Class (
    ClassID INT IDENTITY(1,1) PRIMARY KEY,
    CourseID INT NOT NULL,
    InstructorID INT NOT NULL,
    Time CHAR(5) DEFAULT '09:15',
    Location NVARCHAR(100),
    Enrolled INT DEFAULT 0,
    Limit INT DEFAULT 30,
    FOREIGN KEY (CourseID) REFERENCES Course(CourseID),
    FOREIGN KEY (InstructorID) REFERENCES Instructor(InstructorID)
);

-- Create the Building table
CREATE TABLE Building (
    BuildingID INT IDENTITY(1,1) PRIMARY KEY,
    BuildingName NVARCHAR(100) NOT NULL,
    Address NVARCHAR(200)
);

-- Create the Room table
CREATE TABLE Room (
    RoomID INT IDENTITY(1,1) PRIMARY KEY,
    BuildingID INT NOT NULL,
    RoomNumber NVARCHAR(50) NOT NULL,
    Capacity INT DEFAULT 30,
    FOREIGN KEY (BuildingID) REFERENCES Building(BuildingID)
);

-- Create the Mode of Instruction table
CREATE TABLE ModeOfInstruction (
    ModeID INT IDENTITY(1,1) PRIMARY KEY,
    ModeDescription NVARCHAR(50) NOT NULL
);

-- Create the WorkflowSteps table
CREATE TABLE WorkflowSteps (
    WorkFlowStepKey INT IDENTITY(1,1) PRIMARY KEY,
    WorkFlowStepDescription NVARCHAR(100) NOT NULL,
    WorkFlowStepTableRowCount INT DEFAULT 0,
    StartingDateTime DATETIME2 DEFAULT (SYSDATETIME()),
    EndingDateTime DATETIME2 DEFAULT (SYSDATETIME())
);

Commands completed successfully.

: Msg 2714, Level 16, State 6, Line 6
There is already an object named 'Department' in the database.

Total execution time: 00:00:00.013

# Create Procedures:
- `usp_TrackWorkFlow`: Track Workflow
- `usp_ShowWorkflowSteps`: Show Workflow Steps
- `LoadQueensCourseSchedule`: Load data into Class table

In [2]:
-- Daniel Gargiullo

USE QueensClassSchedule
GO
-- Procedure to track workflow steps
CREATE PROCEDURE usp_TrackWorkFlow
    @WorkFlowStepDescription NVARCHAR(100),
    @RowCount INT
AS
BEGIN
    INSERT INTO WorkflowSteps (WorkFlowStepDescription, WorkFlowStepTableRowCount, StartingDateTime)
    VALUES (@WorkFlowStepDescription, @RowCount, SYSDATETIME());
END;
GO

-- Procedure to show all workflow steps
CREATE PROCEDURE usp_ShowWorkflowSteps
AS
BEGIN
    SELECT * FROM WorkflowSteps;
END;
GO

-- Procedure to load data into the Class table
CREATE PROCEDURE LoadQueensCourseSchedule
AS
BEGIN
    INSERT INTO Class (CourseID, InstructorID, Time, Location, Enrolled, Limit)
    SELECT DISTINCT 
        c.CourseID, 
        i.InstructorID, 
        '09:15' AS DefaultTime,
        'Room 101' AS DefaultLocation,
        20 AS DefaultEnrolled,
        30 AS DefaultLimit
    FROM Course c
    CROSS JOIN Instructor i;
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.016

# Create Procedures:
- `TruncateTables`: Procedure to truncate all tables
- `AddForeignKeys`: Add foreign key constraints
- `RemoveForeignKeys`: Remove foreign key constraints
- `WorkflowSummary`: Summary workflow report

In [1]:
-- Oksana Weigand-Suminski
-- Procedure to truncate all tables (for data reloading)
CREATE PROCEDURE TruncateTables
AS
BEGIN
    TRUNCATE TABLE Class;
    TRUNCATE TABLE Course;
    TRUNCATE TABLE Instructor;
    TRUNCATE TABLE Department;
    TRUNCATE TABLE Building;
    TRUNCATE TABLE Room;
    TRUNCATE TABLE ModeOfInstruction;
    DELETE FROM WorkflowSteps;
END;
GO

-- Procedure to add foreign key constraints
CREATE PROCEDURE AddForeignKeys
AS
BEGIN
    ALTER TABLE Instructor
    ADD CONSTRAINT FK_Instructor_Department FOREIGN KEY (DepartmentID) REFERENCES Department(DepartmentID);

    ALTER TABLE Class
    ADD CONSTRAINT FK_Class_Course FOREIGN KEY (CourseID) REFERENCES Course(CourseID),
        CONSTRAINT FK_Class_Instructor FOREIGN KEY (InstructorID) REFERENCES Instructor(InstructorID);

    ALTER TABLE Room
    ADD CONSTRAINT FK_Room_Building FOREIGN KEY (BuildingID) REFERENCES Building(BuildingID);
END;
GO

-- Procedure to remove foreign key constraints
CREATE PROCEDURE RemoveForeignKeys
AS
BEGIN
    -- Remove Foreign Key from Instructor to Department
    ALTER TABLE Instructor DROP CONSTRAINT FK_Instructor_Department;

    -- Remove Foreign Keys from Class
    ALTER TABLE Class DROP CONSTRAINT FK_Class_Course;
    ALTER TABLE Class DROP CONSTRAINT FK_Class_Instructor;

    -- Remove Foreign Key from Room to Building
    ALTER TABLE Room DROP CONSTRAINT FK_Room_Building;
END;
GO

-- Procedure to perform a summary workflow report
CREATE PROCEDURE WorkflowSummary
AS
BEGIN
    SELECT 
        WorkFlowStepDescription,
        WorkFlowStepTableRowCount,
        StartingDateTime,
        EndingDateTime
    FROM WorkflowSteps
    ORDER BY StartingDateTime;
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.116